In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import os

In [2]:
input_file = 'D:/Calvin/Documents/GTOMSA/Applied Analytics Practicum - CSE 6748/final_data/veteran_data_final.csv'


def load_dataset(file_path):
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        print(f"Dataset '{file_path}' loaded successfully.\n")
        return df
    else:
        raise FileNotFoundError(f"The file '{file_path}' does not exist. Please check the file path.")

# Load the dataset
df = load_dataset(input_file)


Dataset 'D:/Calvin/Documents/GTOMSA/Applied Analytics Practicum - CSE 6748/final_data/veteran_data_final.csv' loaded successfully.



In [3]:
# Display the first five rows of the dataset
print("First five rows of the dataset:")
display(df.head())


First five rows of the dataset:


,sustainer,gender,client_state_rank,all_state_rank,client_zip_rank,all_zip_rank,age,account_id,gift_date,gift_min_t12m,...,drtv_t12m,other_t12m,no_channel_t12m,member_t12m,gift_min_lt,gift_max_lt,gift_avg_lt,gift_count_lt,promo_count_lt,cons_month_lt
0,1,0,1,3,19663,32511,82.0,660793.0,2020-07-04,51.5,...,0,0,0,0,5.0,103.0,30.60,29,27,4
1,1,0,1,3,19663,32511,82.0,660793.0,2020-07-08,51.5,...,0,0,0,0,5.0,103.0,31.30,30,27,4
2,1,0,1,3,19663,32511,82.0,660793.0,2020-04-08,51.5,...,0,0,0,0,5.0,51.5,26.21,26,26,2
3,1,0,1,3,19663,32511,82.0,660793.0,2022-02-15,100.0,...,0,0,0,0,5.0,103.0,34.61,33,28,6
4,1,0,1,3,19663,32511,82.0,660793.0,2022-11-06,100.0,...,0,0,0,0,5.0,103.0,36.53,34,29,6


In [ ]:
# Define the list of columns you want to drop
columns_to_drop = ['all_state_rank','all_zip_rank','tm_t12m','f2f_t12m','events_t12m','no_channel_t12m','member_t12m','gift_date','account_id']

#dropped because 67.5% of age is missing, safer to just remove this feature
# no_channel and member removed as they all have only 1 value

print(f"Columns intended to drop: {columns_to_drop}\n")

existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]

if existing_columns_to_drop:
    print(f"Columns to be dropped (exist in DataFrame): {existing_columns_to_drop}\n")
else:
    print("No specified columns found in the DataFrame to drop.\n")
df = df.drop(columns=existing_columns_to_drop)
print(f"Successfully dropped columns: {existing_columns_to_drop}\n")

print("Remaining columns after dropping:")
print(df.columns.tolist())


Columns intended to drop: ['all_state_rank', 'all_zip_rank', 'tm_t12m', 'f2f_t12m', 'events_t12m', 'no_channel_t12m', 'member_t12m', 'gift_date', 'account_id']

Columns to be dropped (exist in DataFrame): ['all_state_rank', 'all_zip_rank', 'tm_t12m', 'f2f_t12m', 'events_t12m', 'no_channel_t12m', 'member_t12m', 'gift_date', 'account_id']

Successfully dropped columns: ['all_state_rank', 'all_zip_rank', 'tm_t12m', 'f2f_t12m', 'events_t12m', 'no_channel_t12m', 'member_t12m', 'gift_date', 'account_id']

Remaining columns after dropping:
['sustainer', 'gender', 'client_state_rank', 'client_zip_rank', 'age', 'gift_min_t12m', 'gift_max_t12m', 'gift_avg_t12m', 'gift_count_t12m', 'promo_count_t12m', 'cons_month_t12m', 'dm_t12m', 'wm_t12m', 'digital_t12m', 'drtv_t12m', 'other_t12m', 'gift_min_lt', 'gift_max_lt', 'gift_avg_lt', 'gift_count_lt', 'promo_count_lt', 'cons_month_lt']


In [5]:

def handle_missing_values(df, strategy='fill', fill_value=0):
    if strategy == 'fill':
        df_filled = df.fillna(fill_value)
        print(f"Missing values filled with {fill_value}.\n")
        return df_filled
    elif strategy == 'drop':
        df_dropped = df.dropna()
        print("Rows with missing values have been dropped.\n")
        return df_dropped
    else:
        raise ValueError("Unsupported strategy for handling missing values.")

# Handle missing values by filling with -1
df = handle_missing_values(df, strategy='fill', fill_value=-1)

Missing values filled with -1.



In [6]:
#remove_flags = ['dm_t12m','tm_t12m','wm_t12m','digital_t12m','f2f_t12m','events_t12m','drtv_t12m','other_t12m','no_channel_t12m','member_t12m','age','sustainer']
remove_flags = ['dm_t12m','wm_t12m','digital_t12m','drtv_t12m','other_t12m','age','sustainer']
numerical_columns = df.select_dtypes(include=[np.number]).drop(columns=remove_flags).columns
print("Numerical columns:", numerical_columns.tolist())

Numerical columns: ['gender', 'client_state_rank', 'client_zip_rank', 'gift_min_t12m', 'gift_max_t12m', 'gift_avg_t12m', 'gift_count_t12m', 'promo_count_t12m', 'cons_month_t12m', 'gift_min_lt', 'gift_max_lt', 'gift_avg_lt', 'gift_count_lt', 'promo_count_lt', 'cons_month_lt']


Detecting Outliers We'll detect outliers using two methods:

Z-Score Method, Interquartile Range (IQR) Method

In [7]:
# Function to detect outliers using Z-Score
def detect_outliers_zscore(df, numerical_columns, threshold=3):
    outliers = {}
    print("Detecting outliers using Z-Score method\n")
    for column in numerical_columns:
        if df[column].nunique() > 1:
            z_scores = np.abs(stats.zscore(df[column]))
            outliers_indices = np.where(z_scores > threshold)[0]
            outliers[column] = outliers_indices
            print(f"Z-Score Outliers in '{column}': {len(outliers_indices)} detected")
        else:
            outliers[column] = np.array([])
            print(f"Z-Score Outliers in '{column}': 0 detected (constant column)")
    print("\n")
    return outliers

# Detect outliers using Z-Score
outliers_zscore = detect_outliers_zscore(df, numerical_columns)


Detecting outliers using Z-Score method

Z-Score Outliers in 'gender': 11717 detected
Z-Score Outliers in 'client_state_rank': 1548 detected
Z-Score Outliers in 'client_zip_rank': 3824 detected
Z-Score Outliers in 'gift_min_t12m': 1025 detected
Z-Score Outliers in 'gift_max_t12m': 1068 detected
Z-Score Outliers in 'gift_avg_t12m': 1479 detected
Z-Score Outliers in 'gift_count_t12m': 4450 detected
Z-Score Outliers in 'promo_count_t12m': 5866 detected
Z-Score Outliers in 'cons_month_t12m': 9570 detected
Z-Score Outliers in 'gift_min_lt': 227 detected
Z-Score Outliers in 'gift_max_lt': 737 detected
Z-Score Outliers in 'gift_avg_lt': 873 detected
Z-Score Outliers in 'gift_count_lt': 6182 detected
Z-Score Outliers in 'promo_count_lt': 6646 detected
Z-Score Outliers in 'cons_month_lt': 9639 detected




In [8]:
# Function to detect outliers using IQR
def detect_outliers_iqr(df, numerical_columns):
    outliers = {}
    print("Detecting outliers using IQR method...\n")
    for column in numerical_columns:
        Q1 = df[column].quantile(0.10)
        Q3 = df[column].quantile(0.90)
        IQR = Q3 - Q1
        lower_bound = Q1 - 3 * IQR
        upper_bound = Q3 + 3 * IQR
        condition = (df[column] < lower_bound) | (df[column] > upper_bound)
        outliers_indices = df[condition].index
        outliers[column] = outliers_indices
        print(f"IQR Outliers in '{column}': {len(outliers_indices)} detected")
    print("\n")
    return outliers

# Detect outliers using IQR
outliers_iqr = detect_outliers_iqr(df, numerical_columns)

Detecting outliers using IQR method...

IQR Outliers in 'gender': 0 detected
IQR Outliers in 'client_state_rank': 0 detected
IQR Outliers in 'client_zip_rank': 0 detected
IQR Outliers in 'gift_min_t12m': 2264 detected
IQR Outliers in 'gift_max_t12m': 4520 detected
IQR Outliers in 'gift_avg_t12m': 2420 detected
IQR Outliers in 'gift_count_t12m': 458 detected
IQR Outliers in 'promo_count_t12m': 79 detected
IQR Outliers in 'cons_month_t12m': 10360 detected
IQR Outliers in 'gift_min_lt': 1776 detected
IQR Outliers in 'gift_max_lt': 2176 detected
IQR Outliers in 'gift_avg_lt': 2100 detected
IQR Outliers in 'gift_count_lt': 21 detected
IQR Outliers in 'promo_count_lt': 21 detected
IQR Outliers in 'cons_month_lt': 166 detected




Removing Outliers Based on IQR We'll remove the detected outliers using the IQR method to obtain a dataset without outliers

In [9]:
def remove_outliers_iqr(df, numerical_columns):
    print("Removing outliers based on IQR method...\n")
    Q1 = df[numerical_columns].quantile(0.10)
    Q3 = df[numerical_columns].quantile(0.90)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR
    condition = ~((df[numerical_columns] < lower_bound) | (df[numerical_columns] > upper_bound)).any(axis=1)
    df_no_outliers = df[condition]
    print("Outliers removed.")
    print(f"Original dataset size: {df.shape[0]} rows")
    print(f"Dataset without outliers size: {df_no_outliers.shape[0]} rows\n")
    return df_no_outliers

# Remove outliers using IQR
df_without_outliers = remove_outliers_iqr(df, numerical_columns)


Removing outliers based on IQR method...

Outliers removed.
Original dataset size: 314265 rows
Dataset without outliers size: 297084 rows



In [10]:

def display_dataset_without_outliers(df):
    print("Displaying the dataset without outliers:\n")
    # Display the first 5 rows
    print("First five rows of the dataset without outliers:")
    display(df.head())
    
    print("\nSummary Statistics of Dataset Without Outliers:")
    display(df.describe())
    
    # Display the number of rows and columns
    print(f"\nDataset without outliers contains {df.shape[0]} rows and {df.shape[1]} columns.\n")

display_dataset_without_outliers(df_without_outliers)


Displaying the dataset without outliers:

First five rows of the dataset without outliers:


,sustainer,gender,client_state_rank,client_zip_rank,age,gift_min_t12m,gift_max_t12m,gift_avg_t12m,gift_count_t12m,promo_count_t12m,...,wm_t12m,digital_t12m,drtv_t12m,other_t12m,gift_min_lt,gift_max_lt,gift_avg_lt,gift_count_lt,promo_count_lt,cons_month_lt
0,1,0,1,19663,82.0,51.5,103.0,64.38,4,2,...,0,1,0,0,5.0,103.0,30.60,29,27,4
1,1,0,1,19663,82.0,51.5,103.0,61.80,5,2,...,0,1,0,0,5.0,103.0,31.30,30,27,4
2,1,0,1,19663,82.0,51.5,51.5,51.50,1,1,...,0,1,0,0,5.0,51.5,26.21,26,26,2
3,1,0,1,19663,82.0,100.0,100.0,100.00,1,1,...,1,1,0,0,5.0,103.0,34.61,33,28,6
4,1,0,1,19663,82.0,100.0,100.0,100.00,2,2,...,1,1,0,0,5.0,103.0,36.53,34,29,6



Summary Statistics of Dataset Without Outliers:


,sustainer,gender,client_state_rank,client_zip_rank,age,gift_min_t12m,gift_max_t12m,gift_avg_t12m,gift_count_t12m,promo_count_t12m,...,wm_t12m,digital_t12m,drtv_t12m,other_t12m,gift_min_lt,gift_max_lt,gift_avg_lt,gift_count_lt,promo_count_lt,cons_month_lt
count,297084.000000,297084.000000,297084.000000,297084.000000,297084.000000,297084.000000,297084.000000,297084.000000,297084.000000,297084.000000,...,297084.000000,297084.000000,297084.000000,297084.000000,297084.000000,297084.000000,297084.000000,297084.000000,297084.000000,297084.000000
mean,0.566015,0.504359,38.669555,17687.053231,69.107724,18.723100,25.667580,21.607191,5.565349,4.700751,...,0.163317,0.101786,0.007331,0.000114,8.769498,34.491780,17.708261,54.502626,47.321054,8.697368
std,0.495624,0.686523,11.796987,5235.362357,28.084866,16.155662,22.015406,17.758772,5.242833,4.271686,...,0.369656,0.302367,0.085308,0.010697,7.872567,32.357409,12.827375,63.062083,56.284611,13.364865
min,0.000000,0.000000,1.000000,7.000000,-1.000000,0.050000,0.250000,0.250000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.010000,1.000000,0.520000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,32.000000,15224.000000,67.000000,10.000000,10.000000,10.000000,2.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,3.000000,15.000000,9.850000,9.000000,8.000000,2.000000
50%,1.000000,0.000000,42.000000,19322.000000,78.000000,15.000000,20.000000,17.500000,4.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,5.000000,25.000000,14.430000,31.000000,26.000000,3.000000
75%,1.000000,1.000000,48.000000,21734.000000,86.000000,25.000000,29.000000,25.000000,8.000000,6.000000,...,0.000000,0.000000,0.000000,0.000000,10.000000,41.000000,21.830000,79.000000,67.000000,9.000000
max,1.000000,3.000000,51.000000,23219.000000,124.000000,125.000000,170.000000,149.580000,49.000000,41.000000,...,1.000000,1.000000,1.000000,1.000000,70.000000,350.000000,116.430000,482.000000,441.000000,124.000000



Dataset without outliers contains 297084 rows and 22 columns.



In [11]:
def save_dataset_without_outliers(df, output_path):
    try:
        df.to_csv(output_path, index=False)
        print(f"Dataset without outliers saved to '{output_path}'.\n")
    except Exception as e:
        print(f"Error saving file: {e}\n")


output_file = 'D:/Calvin/Documents/GTOMSA/Applied Analytics Practicum - CSE 6748/final_data/veterandata_without_outliers.csv'

# Save the cleaned dataset
save_dataset_without_outliers(df_without_outliers, output_file)


Dataset without outliers saved to 'D:/Calvin/Documents/GTOMSA/Applied Analytics Practicum - CSE 6748/final_data/veterandata_without_outliers.csv'.

